In [1]:
pip install nltk scikit-learn streamlit

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/e4/08/c962e38f4450dafb98af49f8988a736dd67ae9cc26a5704c43269f506bc8/streamlit-1.42.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/aa/f3/0b6ced594e51cc95d8c1fc1640d3623770d01e4969d29c0bd09945fafefa/altair-5.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/10/cb/f2ad4230dc2eb1a74edf38f1a38b9b52277f75bef262d8908e60d957e13c/blinker-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/72/76/20fa66124dbe6be5cafeb312ece67de6b61dd91a0247d1ea13db4ebb33c2/cachetools-5.5.2-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf<6,>=3.20 from https://files.pythonhosted.org/packages/61/fa/aae8e10512b83de633f2646506a6d835b151edf4b30d18d73afd01447

In [2]:
# Importing necessary libraries
import nltk
import random
import os
import ssl
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cheta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
intents = [
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "What’s up", "How are you"],
        "responses": ["Hi there!", "Hello! How can I assist you today?", "Hey! Welcome to Stufin.", "What’s up? Need help with something?", "I’m doing great, thanks! How about you?"]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "See you later", "Goodbye", "Take care"],
        "responses": ["Goodbye! Happy shopping!", "See you later!", "Take care, and come back soon!"]
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it"],
        "responses": ["You’re welcome!", "No problem, glad to help!", "Happy to assist you!"]
    },
    {
        "tag": "about",
        "patterns": ["What can you do", "Who are you", "What are you", "What is Stufin"],
        "responses": ["I’m Stufin’s chatbot, here to help you shop for IoT devices, troubleshoot issues, and more!", "Stufin is your go-to IoT e-commerce platform, and I’m here to assist with your queries!", "I can help you browse products, check orders, and set up your IoT devices."]
    },
    {
        "tag": "help",
        "patterns": ["Help", "I need help", "Can you help me", "What should I do"],
        "responses": ["Of course! What do you need help with?", "I’m here for you—what’s the issue?", "How can I assist you today?"]
    },
    {
        "tag": "browse_products",
        "patterns": ["Show me products", "What do you sell", "Browse IoT devices", "What’s available"],
        "responses": ["We’ve got a range of IoT devices—smart lights, cameras, thermostats, and more! What are you looking for?", "You can browse our IoT collection! Want to see smart home gadgets or something specific?", "Check out our IoT products—what category interests you?"]
    },
    {
        "tag": "search_product",
        "patterns": ["Do you have a smart thermostat", "Find me a Wi-Fi camera", "Looking for a smart lock", "Search for IoT lights"],
        "responses": ["Let me check that for you! One moment.", "I’ll find that smart device—give me a sec!", "Searching for your IoT item now!"]
    },
    {
        "tag": "product_details",
        "patterns": ["Tell me about the smart speaker", "What does this device do", "Features of the IoT camera", "How does this work"],
        "responses": ["I’ll give you the rundown on that device!", "Here’s what that IoT gadget can do for you.", "Let me share the features—stay tuned!"]
    },
    {
        "tag": "check_price",
        "patterns": ["How much is the smart light", "What’s the price", "Is this expensive", "Any discounts"],
        "responses": ["I’ll get you the latest price on that!", "Here’s the cost—any budget in mind?", "Let me check for discounts too!"]
    },
    {
        "tag": "add_to_cart",
        "patterns": ["Add this to my cart", "Put the smart plug in my cart", "I want to buy this", "Add it"],
        "responses": ["Added to your cart!", "It’s in your cart now—anything else?", "Done! What’s next on your list?"]
    },
    {
        "tag": "track_order",
        "patterns": ["Where’s my order", "Track my package", "When will it arrive", "Order status"],
        "responses": ["Let me check your order status!", "I’ll track that for you—order number handy?", "Your IoT device is on its way—let me update you!"]
    },
    {
        "tag": "device_setup",
        "patterns": ["How do I set up my device", "Install my smart lock", "Setup instructions", "Help me connect it"],
        "responses": ["I’ll guide you through the setup!", "Setting up your IoT device? Here’s how.", "Let’s get that connected—step by step!"]
    },
    {
        "tag": "troubleshooting",
        "patterns": ["My device isn’t working", "Smart light won’t connect", "Why is it offline", "Fix my IoT camera"],
        "responses": ["Let’s troubleshoot that together—what’s happening?", "I’ll help you fix it—describe the issue.", "No worries, we’ll get it back online!"]
    },
    {
        "tag": "compatibility",
        "patterns": ["Does it work with Alexa", "Is this compatible with my hub", "Will it sync with Google Home", "Device compatibility"],
        "responses": ["I’ll check compatibility for you!", "Let me confirm if it works with your setup.", "Good question—let’s see if it syncs!"]
    },
    {
        "tag": "shipping_info",
        "patterns": ["How long does shipping take", "What’s the shipping cost", "Do you ship to my location", "Delivery time"],
        "responses": ["Shipping details coming right up!", "I’ll get you the latest on shipping costs and times.", "Let me check delivery options for you!"]
    },
    {
        "tag": "returns",
        "patterns": ["How do I return this", "Can I get a refund", "Return policy", "Send it back"],
        "responses": ["Here’s how returns work at Stufin!", "I’ll explain the refund process—ready?", "Let’s get that return started for you!"]
    },
    {
        "tag": "contact_support",
        "patterns": ["Talk to someone", "Customer service", "I need a human", "Support contact"],
        "responses": ["I’ll connect you to our support team!", "Need a human touch? Here’s how to reach us.", "Let me get you to customer service!"]
    },
    {
        "tag": "recommendations",
        "patterns": ["What should I buy", "Recommend an IoT device", "Best smart home gadget", "Suggest something"],
        "responses": ["I’ve got some great IoT picks for you!", "How about a smart device for your home? Any preferences?", "Let me suggest something tailored for you!"]
    }
]

In [9]:
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

#Preprocess the data
tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

In [12]:
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x, y)

LogisticRegression(max_iter=10000, random_state=0)

In [13]:
def chatbot(input_text):
    input_text = vectorizer.transform([input_text])
    tag = clf.predict(input_text)[0]
    for intent in intents:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
            return response

In [15]:
user_input = "What are you?"
response = chatbot(user_input)
print(response)

Stufin is your go-to IoT e-commerce platform, and I’m here to assist with your queries!


In [16]:
user_input = "Show me products"
response = chatbot(user_input)
print(response)

Check out our IoT products—what category interests you?


In [17]:
user_input = "What is Stufin"
response = chatbot(user_input)
print(response)

I’m Stufin’s chatbot, here to help you shop for IoT devices, troubleshoot issues, and more!
